# Project Title
### Data Engineering Capstone Project

#### Project Summary
Twitter data posted about Amazon, Apple, Google, Microsoft, and Tesla, and market value data of these companies will be used to create a database that is optimized to query and analyze. An ETL pipeline is going to be built to create the database.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import numpy as np
import pandas as pd
import datetime
import psycopg2
from sql_queries import fact_tweet_table_insert, user_table_insert, company_table_insert, value_table_insert, time_table_insert

### Step 1: Scope the Project and Gather Data

#### Scope 
A company's market values are affected by different factors. One of the important factors could be the public opinion about that company. This project ist going to analyze the correlation between the market value of company respect to the public opinion of that company. Particularly, this task will try to understand whether negative tweets about companie affect their market values.

#### Describe and Gather Data

##### Twitter Data
This dataset as a part of the paper published in the 2020 IEEE International Conference on Big Data under the 6th Special Session on Intelligent Data Mining track. The dataset contains over 3 million unique tweets with their information such as tweet id, author of the tweet, post date, the text body of the tweet, and the number of comments, likes, and retweets of tweets matched with the related company.

##### Company Value Data
This dataset contains daily OPEN, CLOSE, VOLUME, HIGH, and LOW values of Amazon, Apple, Google, Microsoft, and Tesla companies as tagged by dates. Values are fetched from the official NASDAQ website.


###### Twitter Dataset

In [2]:
# Read in the data here
path = ""
tweet = pd.read_csv("Tweet.csv")
pd.set_option('display.max_columns', 15)

In [3]:
tweet.head(10)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
5,550443808606126081,aaplstocknews,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
6,550443809700851716,iknowfirst,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
7,550443857142611968,Cprediction,1420071016,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
8,550443857595600896,iknowfirst_br,1420071017,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
9,550443857692078081,Gold_prediction,1420071017,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


###### Market price Data

In [4]:
price = pd.read_csv("CompanyValues.csv")

In [5]:
price.head(10)
#len(price)
#len(price.volume.unique())

,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value
0,AAPL,2020-05-29,317.94,38399530,319.25,321.15,316.4700
1,AAPL,2020-05-28,318.25,33449100,316.77,323.44,315.6300
2,AAPL,2020-05-27,318.11,28236270,316.14,318.71,313.0900
3,AAPL,2020-05-26,316.73,31380450,323.50,324.24,316.5000
4,AAPL,2020-05-22,318.89,20450750,315.77,319.23,315.3500
5,AAPL,2020-05-21,316.85,25672210,318.66,320.89,315.8700
6,AAPL,2020-05-20,319.23,27876220,316.68,319.52,316.2000
7,AAPL,2020-05-19,313.14,25432390,315.03,318.52,313.0100
8,AAPL,2020-05-18,314.96,33843130,313.17,316.50,310.3241
9,AAPL,2020-05-15,307.71,41587090,300.35,307.90,300.2100


### Step 2: Explore and Assess the Data
#### Explore the Data 

first doing some EDA

In [6]:
# EDA
print("Twitter Data")
twitter_count=tweet.count()
print(f"Rows: {twitter_count}")
print(f"Columns: {len(tweet.columns)}")
print()
print("Market Price")
mrkt_price_count=price.count()
print(f"Rows: {mrkt_price_count}")
print(f"Columns: {len(tweet.columns)}")

Twitter Data
Rows: tweet_id       3717964
writer         3670691
post_date      3717964
body           3717964
comment_num    3717964
retweet_num    3717964
like_num       3717964
dtype: int64
Columns: 7

Market Price
Rows: ticker_symbol    17528
day_date         17528
close_value      17528
volume           17528
open_value       17528
high_value       17528
low_value        17528
dtype: int64
Columns: 7


#### Cleaning Steps
Document steps necessary to clean the data
- convert the post_date column to date
- check for missing values
- check for duplicates values and drop them
- add company symbol ticker to tweet data
- create unique IDs for writer and symbol ticker columns

In [7]:
# convert the post_date column to date type
tweet['post_date'] = pd.to_datetime(tweet['post_date'], unit='s')
#tweet['date'].head()

time_data = []
for element in tweet['post_date']:
    time_data.append([element, element.hour, element.day, element.week, element.month, element.year, element.day_name()])
column_labels = ('date_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.DataFrame(time_data, columns=column_labels)
time_df.head()

,date_time,hour,day,week,month,year,weekday
0,2015-01-01 00:00:57,0,1,1,1,2015,Thursday
1,2015-01-01 00:01:36,0,1,1,1,2015,Thursday
2,2015-01-01 00:01:50,0,1,1,1,2015,Thursday
3,2015-01-01 00:06:47,0,1,1,1,2015,Thursday
4,2015-01-01 00:10:05,0,1,1,1,2015,Thursday


In [8]:
# check for null values in tweet data
tweet.isnull().sum().sort_values(ascending = False)

writer         47273
tweet_id           0
post_date          0
body               0
comment_num        0
retweet_num        0
like_num           0
dtype: int64

The "writer" column is the only column with missing data. Since this column is not going to be used in the analysis, I am going to ignore the missings for this column for now.

In [9]:
# check for null values in the market price data
price.isnull().sum().sort_values(ascending = False)

ticker_symbol    0
day_date         0
close_value      0
volume           0
open_value       0
high_value       0
low_value        0
dtype: int64

In [10]:
# checking for duplicates values in the tweet data
tweet_duplicates = tweet[tweet["tweet_id"].duplicated()]
len(tweet_duplicates)

0

##### load the company data that contains company symbol, and merge this data with the tweet dataframe

In [11]:
company_symbol = pd.read_csv('Company_Tweet.csv')
# check for any duplicates in
duplicates_company_symbol = company_symbol[company_symbol["tweet_id"].duplicated()]
len(duplicates_company_symbol) # 618481 duplicates
# drop duplicates
company_symbol.drop_duplicates(subset = ["tweet_id"], inplace = True)

In [12]:
# merge company symbol to the tweet data
tweets = pd.merge(tweet,company_symbol,on='tweet_id',how='inner')
tweets.shape

(3717964, 8)

In [13]:
# count unique values of ticker symbol 
print("Twitter Data")
print(tweets.ticker_symbol.value_counts())
print()
print("Market Price Data")
price.ticker_symbol.value_counts()

Twitter Data
AAPL     1425013
TSLA      958370
AMZN      534008
GOOG      312590
MSFT      268257
GOOGL     219726
Name: ticker_symbol, dtype: int64

Market Price Data


AAPL     3085
MSFT     3085
AMZN     3085
GOOGL    3085
TSLA     3065
GOOG     2123
Name: ticker_symbol, dtype: int64

##### Create unique IDs for writer and symbol ticker column

In [14]:
# create ids for writer column
np.random.seed(1)
names = tweets['writer'].unique().tolist()
ids = np.random.randint(low=1e9, high=1e10, size = len(names))
len(set(ids))
maps = {k:v for k,v in zip(names, ids)}
tweets['writer_id'] = tweets['writer'].map(maps)

# create ids for ticker_symbol column
np.random.seed(1)
symbol = tweets['ticker_symbol'].unique().tolist()
sym_id = np.random.randint(low=1e9, high=1e10, size = len(symbol))
len(set(sym_id))
mappings = {k:v for k,v in zip(symbol, sym_id)}
tweets['ticker_symbol_id'] = tweets['ticker_symbol'].map(mappings)

tweets.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,ticker_symbol,writer_id,ticker_symbol_id
0,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,9577843435,9577843435
1,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,5005303368,9577843435
2,550441732014223360,DozenStocks,2015-01-01 00:01:50,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,AMZN,2703301249,5005303368
3,550442977802207232,ShowDreamCar,2015-01-01 00:06:47,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,TSLA,2666063943,2703301249
4,550443807834402816,i_Know_First,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,8171504636,9577843435


##### giving the same ticker symbol ID to the ticker symbol in price dataset 

In [15]:
# create ids for ticker_symbol column in price dataset
mappings = {k:v for k,v in zip(tweets["ticker_symbol"].unique(), tweets["ticker_symbol_id"].unique())}
price['ticker_symbol_id'] = price['ticker_symbol'].map(mappings)
price.groupby("ticker_symbol")["ticker_symbol_id"].unique()

ticker_symbol
AAPL     [9577843435]
AMZN     [5005303368]
GOOG     [2666063943]
GOOGL    [7590257550]
MSFT     [8171504636]
TSLA     [2703301249]
Name: ticker_symbol_id, dtype: object

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [18]:
# After running create_tables.py, insert the data into the database
conn = psycopg2.connect("dbname=capstone user=postgres password=password")
cur = conn.cursor()

In [19]:
df_tweet = tweets[["tweet_id", "post_date", "writer_id", "ticker_symbol_id", "body", "comment_num", "retweet_num", "like_num"]]

for index, row in df_tweet.iterrows():
    cur.execute(fact_tweet_table_insert, list(row.values))
    conn.commit()

In [20]:
df_user = tweets[["writer_id", "writer"]]

for index, row in df_user.iterrows():
    cur.execute(user_table_insert, list(row.values))
    conn.commit()

In [21]:
df_company = tweets[["ticker_symbol_id", "ticker_symbol"]]
df_company = df_company.drop_duplicates(subset = ["ticker_symbol_id"])

for index, row in df_company.iterrows():
    cur.execute(company_table_insert, list(row.values))
    conn.commit()

In [22]:
df_value = price[["day_date", "volume", "open_value", "high_value", "low_value", "ticker_symbol_id"]]


for index, row in df_value.iterrows():
    cur.execute(value_table_insert, list(row.values))
    conn.commit()

In [24]:
df_time = time_df[["date_time", "hour", "day", "week", "month", "year", "weekday"]]

for index, row in df_time.iterrows():
    cur.execute(time_table_insert, list(row.values))
    conn.commit()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [26]:
# Perform quality checks here
cur.execute("SELECT COUNT(*) FROM tweets")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table tweets")
    
cur.execute("SELECT COUNT(*) FROM users")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table users")
    
cur.execute("SELECT COUNT(*) FROM time")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table time")
    
cur.execute("SELECT COUNT(*) FROM value")
conn.commit()
if cur.rowcount < 1:
    print("No data found in table value")

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.